In [ ]:
!pip install trl peft accelerate bitsandbytes

In [18]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

base_model = "m-polignano-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)



Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
messages = [
    {"role": "system", "content": "You are an AI assistant! Answer the following questions in English."},
    {"role": "user", "content": "Why the sky is blue?"}
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, fast=False)
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
for k,v in inputs.items():
    inputs[k] = v.cuda()
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.6)
results = tokenizer.batch_decode(outputs)[0]
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer in a clear way!<|eot_id|><|start_header_id|>user<|end_header_id|>

Why the sky is blue?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sky is blue because of the scattering of sunlight by the Earth's atmosphere. The particles in the atmosphere, such as nitrogen and oxygen, scatter the sunlight in all directions, creating a blue color. The blue color is also caused by the presence of water vapor in the atmosphere, which can also scatter the sunlight. <|eot_id|>


In [5]:
tokenizer.chat_template

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"

In [20]:
import transformers
pipe = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # langchain expects the full text
    task='text-generation',
    max_new_tokens=512, # max number of tokens to generate in the output
    temperature=0.8  #temperature for more or less creative answers
)
messages = [
    {"role": "system", "content": "Sei un an assistente AI per la lingua Italiana di nome LLaMAntino-3 ANITA \
    (Advanced Natural-based interaction for the ITAlian language). \
    Rispondi nella lingua usata per la domanda in modo chiaro, semplice ed esaustivo. "},
    {"role": "user", "content": "Quanto fa 5469+7472?"}
]
# Method 1
sequences = pipe(messages)
for seq in sequences:
    print(f"{seq['generated_text']}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


5469 + 7472 = 12941. 
